In [69]:
import numpy as np
from collections import Counter

In [2]:
scanners = []
with open('sample.txt') as f:
        lines = list(map(lambda x: x.split('\n'), f.read().split('\n\n')))
    for line in lines:
        line.pop(0)
        intarr = [list(map(int, el.split(','))) for el in line]
        scanners.append(np.array(intarr))

In [3]:
base = scanners[0]
scanners = scanners[1:]

In [4]:
x_perms = [[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 1, 0], [2, 0, 1]]
yz_perms = [[1,1,1], [1,-1,1], [1,1,-1], [1,-1,-1], [-1,1,1], [-1,-1,1], [-1,1,-1], [-1,-1,-1]]

In [5]:
scanner_perms = []
for scanner in scanners:
    perms = []
    for axes in x_perms:
        for multi in yz_perms:
            perms.append(np.multiply(scanner[:, axes], multi))
    scanner_perms.append(perms)

In [12]:
scan1 = scanners[0]

In [239]:
scan1_dists = []
scan1_dist_done = []

In [240]:
for i, point1 in enumerate(scan1):
    for j, point2 in enumerate(scan1):
        if sum(point1) != sum(point2) and {i, j} not in scan1_dist_done:
            scan1_dists.append((np.linalg.norm(point1 - point2), i, j))
            scan1_dist_done.append({i, j})

In [241]:
base_dists = []
base_dist_done = []
for i, point1 in enumerate(base):
    for j, point2 in enumerate(base):
        if sum(point1) != sum(point2) and {i, j} not in base_dist_done:
            base_dists.append((np.linalg.norm(point1 - point2), i, j))
            base_dist_done.append({i, j})

In [242]:
scan1_dist_dict = {d: (i, j) for d, i, j in scan1_dists}
base_dist_dict = {d: (i, j) for d, i, j in base_dists}

In [306]:
common_dists = set(scan1_dist_dict.keys()).intersection(base_dist_dict.keys())
len(common_dists)

66

In [244]:
translation = {}
for d in common_dists:
    translation[base_dist_dict[d]] = scan1_dist_dict[d]

In [314]:
common_nodes = set()
for n1, n2 in translation.keys():
    common_nodes.add(n1); common_nodes.add(n2)
set(chain(*[[n1, n2] for n1, n2 in translation.keys()]))

{2, 6, 8, 13, 15, 16, 18, 19, 21, 22, 23, 24}

In [246]:
trans = {i: Counter() for i in common_nodes}
for (b1, b2), (s1, s2) in translation.items():
    trans[b1][s1] += 1
    trans[b1][s2] += 1
    trans[b2][s1] += 1
    trans[b2][s2] += 1

In [247]:
final_trans = {}
for i, dic in trans.items():
    final_trans[i] = max(list(dic.items()), key=lambda x: x[1])[0]

In [248]:
final_trans

{0: 3,
 1: 8,
 3: 12,
 4: 1,
 5: 24,
 6: 18,
 7: 10,
 9: 0,
 12: 2,
 14: 5,
 19: 15,
 24: 19}

In [249]:
n1, n2 = list(final_trans.keys())[:2]

In [250]:
base_diffs = base[n1] - base[n2]
base_diffs

array([ -124,    55, -1310])

In [251]:
scan_diffs = scan1[final_trans[n1]] - scan1[final_trans[n2]]
scan_diffs

array([ -124,    55, -1310])

In [252]:
abs(base_diffs) - abs(scan_diffs) > 0

array([False, False, False])

In [253]:
minusmask = base_diffs + scan_diffs == 0
rot = np.where(minusmask, 1, -1)

In [254]:
scan1_rot = np.multiply(scan1, rot)

In [256]:
base[0] + scan1_rot[3]

array([0, 0, 0])

In [257]:
scan4 = scanners[3]

In [273]:
scan4_dists = []
scan4_dist_done = []
for i, point1 in enumerate(scan4):
    for j, point2 in enumerate(scan4):
        if sum(point1) != sum(point2) and {i, j} not in scan4_dist_done:
            scan4_dists.append((np.linalg.norm(point1 - point2), i, j))
            scan4_dist_done.append({i, j})

In [274]:
scan4_dist_dict = {d: (i, j) for d, i, j in scan4_dists}
common_dists = set(scan1_dist_dict.keys()).intersection(set(scan4_dist_dict.keys()))
translation = {}
for d in common_dists:
    translation[scan1_dist_dict[d]] = scan4_dist_dict[d]
common_nodes = set()
for n1, n2 in translation.keys():
    common_nodes.add(n1); common_nodes.add(n2)
trans = {i: Counter() for i in common_nodes}
for (b1, b2), (s1, s2) in translation.items():
    trans[b1][s1] += 1
    trans[b1][s2] += 1
    trans[b2][s1] += 1
    trans[b2][s2] += 1
final_trans = {}
for i, dic in trans.items():
    final_trans[i] = max(list(dic.items()), key=lambda x: x[1])[0]

In [275]:
n1, n2 = list(final_trans.items())[0]

In [276]:
base_diffs = abs(scan1[2] - scan1[6])
scan_diffs = abs(scan4[final_trans[2]] - scan4[final_trans[6]])

In [277]:
base_diffs

array([ 855, 1486,  485])

In [278]:
scan_diffs

array([ 485,  855, 1486])

In [279]:
axis_perms = [[0,1,2],[0,2,1],[1,0,2],[1,2,0],[2,0,1],[2,1,0]]
diff_perms = []
for axes in axis_perms:
    diff_perms.append(scan_diffs[axes])

In [301]:
for i, perm in enumerate(diff_perms):
    if np.allclose(perm, base_diffs):
        scan_diffs = perm
        scan4_rot = scan4[:, axis_perms[i]]

In [293]:
base_diffs + scan_diffs

array([1710, 2972,  970])

In [303]:
minusmask = base_diffs + scan_diffs == 0
rot = np.where(minusmask, 1, -1)
scan_rot = np.multiply(scan4_rot, rot)

In [304]:
base_diffs[2]-scan_rot[final_trans[2]]

array([ 912, -319, -258])

In [296]:
base_scan1 = np.array([68, -1246,   -43])

In [297]:
base_diffs

array([ 855, 1486,  485])

In [213]:
scan1_based = scan1_rot + base_scan1

In [162]:
-618,-824,-621
-537,-823,-458
-447,-329,318
404,-588,-901
544,-627,-890
528,-643,409
-661,-816,-575
390,-675,-793
423,-701,434
-345,-311,381
459,-707,401
-485,-357,347

array([[ 404, -588, -901],
       [ 528, -643,  409],
       [-838,  591,  734],
       [ 390, -675, -793],
       [-537, -823, -458],
       [-485, -357,  347],
       [-345, -311,  381],
       [-661, -816, -575],
       [-876,  649,  763],
       [-618, -824, -621],
       [ 553,  345, -567],
       [ 474,  580,  667],
       [-447, -329,  318],
       [-584,  868, -557],
       [ 544, -627, -890],
       [ 564,  392, -477],
       [ 455,  729,  728],
       [-892,  524,  684],
       [-689,  845, -530],
       [ 423, -701,  434],
       [   7,  -33,  -71],
       [ 630,  319, -379],
       [ 443,  580,  662],
       [-789,  900, -551],
       [ 459, -707,  401]])

array([[ -618,  -824,  -621],
       [ -537,  -823,  -458],
       [ -447,  -329,   318],
       [  404,  -588,  -901],
       [  -27, -1108,   -65],
       [  544,  -627,  -890],
       [  408, -1815,   803],
       [ -499, -1607,  -770],
       [  528,  -643,   409],
       [ -601, -1648,  -643],
       [ -661,  -816,  -575],
       [  568, -2007,  -577],
       [  390,  -675,  -793],
       [  534, -1912,   768],
       [  497, -1838,  -617],
       [  423,  -701,   434],
       [ -635, -1737,   486],
       [  396, -1931,  -563],
       [ -345,  -311,   381],
       [  459,  -707,   401],
       [ -518, -1681,  -600],
       [  432, -2009,   850],
       [ -739, -1745,   668],
       [ -687, -1600,   576],
       [ -485,  -357,   347]])

array([[ 404, -588, -901],
       [ 528, -643,  409],
       [-838,  591,  734],
       [ 390, -675, -793],
       [-537, -823, -458],
       [-485, -357,  347],
       [-345, -311,  381],
       [-661, -816, -575],
       [-876,  649,  763],
       [-618, -824, -621],
       [ 553,  345, -567],
       [ 474,  580,  667],
       [-447, -329,  318],
       [-584,  868, -557],
       [ 544, -627, -890],
       [ 564,  392, -477],
       [ 455,  729,  728],
       [-892,  524,  684],
       [-689,  845, -530],
       [ 423, -701,  434],
       [   7,  -33,  -71],
       [ 630,  319, -379],
       [ 443,  580,  662],
       [-789,  900, -551],
       [ 459, -707,  401]])

In [316]:
0 in np.array([2,5,7,0])

True